In [1]:
#Mcallen
#San Antonio
#Edinburg
#Austin
#Dallas
#San Marcos
#Corpus Christi
#Weslaco
#Boerne
#Houston

### Imports

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import requests
import pandas as pd
import urllib.parse

### HTTP Request

In [2]:
#store website in a variable
website = 'https://www.trulia.com/TX/Dallas/'

In [3]:
#set up selenium webdriver
driver = webdriver.Chrome()

#get the webpage using selenium

try:
    # Get the webpage using Selenium
    driver.get(website)
    time.sleep(5)  # Give the page some time to load

    # Scroll down to load more content
    for _ in range(20):
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
        time.sleep(1)  # Adjust the delay based on the page loading speed

    # Get the updated page source after scrolling
    updated_html = driver.page_source

    # Create a BeautifulSoup object using the updated HTML
    soup = BeautifulSoup(updated_html, 'html.parser')

    # Extract the desired results with data-testid
    result_container = soup.find_all(attrs={'data-testid': True})


    # Retrieve anchor tags into a list and establish base URL
    url_part_1 = 'https://www.trulia.com'
    url_part_2 = []

    for item in result_container:
        for link in item.find_all('div', {'data-testid': 'property-card-details'}):
            url_part_2.append(link.find('a').get('href'))

finally:
    driver.quit()  # Close the browser window when done

In [4]:
# Remove spaces and duplicates from list
url_only = [x.strip(' ') for x in url_part_2]
url_only = list(dict.fromkeys(url_only))
len(url_only)

40

In [5]:
# Concatenate URL
url_joined = []
for link_2 in url_only:
    url_joined.append(urllib.parse.urljoin(url_part_1,link_2))

In [6]:
url_joined

['https://www.trulia.com/home/2535-wedglea-dr-106-dallas-tx-75211-26934562',
 'https://www.trulia.com/home/111-s-tillery-st-dallas-tx-75211-26879605',
 'https://www.trulia.com/home/3704-prague-mews-dallas-tx-75236-84001725',
 'https://www.trulia.com/home/12058-lueders-ln-dallas-tx-75230-53390010',
 'https://www.trulia.com/home/3321-gus-thomasson-rd-dallas-tx-75228-26843015',
 'https://www.trulia.com/home/10511-longmeadow-dr-dallas-tx-75238-26854170',
 'https://www.trulia.com/home/17212-village-ln-dallas-tx-75248-26877345',
 'https://www.trulia.com/builder-community-plan/Tenison-Village-At-Buckner-Terrace-Makena-2055886390',
 'https://www.trulia.com/home/7135-elliott-dr-dallas-tx-75227-26785100',
 'https://www.trulia.com/home/4242-lomo-alto-dr-n58-dallas-tx-75219-53390296',
 'https://www.trulia.com/home/2534-gladstone-dr-dallas-tx-75211-26713129',
 'https://www.trulia.com/home/9307-sugarberry-rd-dallas-tx-75249-26892331',
 'https://www.trulia.com/home/6120-velasco-ave-dallas-tx-75214-26

### Get data from links

In [7]:
# address
# bedrooms
# bathrooms
# sqft
# year built
# parking
# price

In [8]:
first_link = url_joined[0]
print(first_link)

https://www.trulia.com/home/2535-wedglea-dr-106-dallas-tx-75211-26934562


### Get request and Soup Object

In [9]:
response = requests.get(first_link, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'})

In [10]:
soup = BeautifulSoup(response.content, 'html.parser')

In [1]:
# Commenting this step out to avoid copyright
#soup

#### Put all of the attributes together and loop through results from page 1


In [12]:
address = []
bedrooms = []
bathrooms = []
area = []
year_built = []
parking = []
price = []

# Loop through all joined links
for link in url_joined:
    response = requests.get(link, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'})
    
    # Create soup object
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Address
    try:
        address.append(soup.find('span', {'data-testid':'home-details-summary-headline'}).get_text())
    except:
        address.append('')
    # Bedrooms  
    try:
        bedrooms.append(soup.find('li', {'data-testid':'bed'}).get_text())
    except:
        bedrooms.append('')
        
    # Bathrooms
    try:
        bathrooms.append(soup.find('li', {'data-testid':'bath'}).get_text())
    except:
        bathrooms.append('')
        
    # Square Feet
    try:
        area.append(soup.find('li', {'data-testid':'floor'}).get_text())
    except:
        area.append('')
    
    # Year Built
    try:
        year_built.append(soup.find('div',string='Year Built').findNext('span').get_text())
        year_built = [x.strip('Year Built: ') for x in year_built] 
    except:
        year_built.append('')
    
    # Parking
    try:
        parking.append(soup.find(string='Parking').findNext('div').get_text())
    except:
        parking.append('')
    
    # Price
    try:
        price.append(soup.find('h3', {'data-testid':'on-market-price-details'}).get_text())
    except:
        price.append('') 
    
    # Create a dictionary with results
    output = {'Address': address, 'Bathrooms': bathrooms, 'Area':area, 'Year Built':year_built, 
              'Parking':parking, 'Price':price}
    
output

{'Address': ['2535 Wedglea Dr #106',
  '111 S  Tillery St',
  '3704 Prague Mews',
  '12058 Lueders Ln',
  '3321 Gus Thomasson Rd',
  '10511 Longmeadow Dr',
  '17212 Village Ln',
  'Makena Plan in Tenison Village at Buckner Terrace by Mattamy Homes',
  '7135 Elliott Dr',
  '4242 Lomo Alto Dr #N58',
  '2534 Gladstone Dr',
  '9307 Sugarberry Rd',
  '6120 Velasco Ave',
  '10525 Andover Dr',
  '2555 N  Pearl St #1701',
  '1202 N  Clinton Ave',
  '6368 Bordeaux Ave',
  '17916 Castle Bend Dr',
  '348 Kahala Dr',
  '2514 W  10th St',
  '12921 Abrams Rd #401',
  '1505 Elm St #405',
  '3705 Patience Blvd E  #E',
  '3910 Candlenut Ln',
  '10308 Lavano Ln',
  '4848 Thunder Rd',
  '9018 Longmont Dr',
  '927 W  12th St',
  '7116 Wake Forrest Dr',
  '3918 Fairfax Ave',
  '7812 Meadow Park Dr #228',
  '4505 Crown Knoll Cir',
  '14275 Greenhaw Ln',
  '4206 Cabell Dr #102',
  '9147 Cochran Heights Dr',
  '7807 W  Lovers Ln',
  '4218 Gilbert Ave #G',
  '3531 Gallagher St',
  '1312 Valley St',
  '6984 San

In [13]:
# Create dataframe
df = pd.DataFrame(output)
df

,Address,Bathrooms,Area,Year Built,Parking,Price
0,2535 Wedglea Dr #106,1 Bath,726 sqft,1983,Open Parking,"$215,000"
1,111 S Tillery St,2 Baths,"1,276 sqft",2000,Garage,"$285,000"
2,3704 Prague Mews,2 Baths,"1,712 sqft",2007,2 Car Garage,"$324,999"
3,12058 Lueders Ln,3 Baths,"1,958 sqft",1999,2 Car Garage,"$980,000"
4,3321 Gus Thomasson Rd,2 Baths,"1,679 sqft",2020,1 Car Garage,"$379,000"
5,10511 Longmeadow Dr,2 Baths,"1,855 sqft",1957,2 Car Garage,"$599,000"
6,17212 Village Ln,4 Baths,"4,465 sqft",1984,2 Car Garage,"$1,500,000"
7,Makena Plan in Tenison Village at Buckner Terr...,4 Baths,"2,410 sqft",2024,Garage,"$477,720+"
8,7135 Elliott Dr,3 Baths,"2,052 sqft",1977,2 Car Garage,"$399,000"
9,4242 Lomo Alto Dr #N58,4 Baths,"2,512 sqft",1950,2 Car Garage,"$850,000"


In [14]:
# Add location to dataframe
df['Location'] = 'Dallas'
df

,Address,Bathrooms,Area,Year Built,Parking,Price,Location
0,2535 Wedglea Dr #106,1 Bath,726 sqft,1983,Open Parking,"$215,000",Dallas
1,111 S Tillery St,2 Baths,"1,276 sqft",2000,Garage,"$285,000",Dallas
2,3704 Prague Mews,2 Baths,"1,712 sqft",2007,2 Car Garage,"$324,999",Dallas
3,12058 Lueders Ln,3 Baths,"1,958 sqft",1999,2 Car Garage,"$980,000",Dallas
4,3321 Gus Thomasson Rd,2 Baths,"1,679 sqft",2020,1 Car Garage,"$379,000",Dallas
5,10511 Longmeadow Dr,2 Baths,"1,855 sqft",1957,2 Car Garage,"$599,000",Dallas
6,17212 Village Ln,4 Baths,"4,465 sqft",1984,2 Car Garage,"$1,500,000",Dallas
7,Makena Plan in Tenison Village at Buckner Terr...,4 Baths,"2,410 sqft",2024,Garage,"$477,720+",Dallas
8,7135 Elliott Dr,3 Baths,"2,052 sqft",1977,2 Car Garage,"$399,000",Dallas
9,4242 Lomo Alto Dr #N58,4 Baths,"2,512 sqft",1950,2 Car Garage,"$850,000",Dallas


In [15]:
df.to_excel('dataframe_dallas.xlsx', index=False)